In [ ]:
import sys
sys.path.insert(0,'..')
from wavhandler import *
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from utils_train import train_test_val_split, TrainConfiguration, train_generator
from utils_train import valid_generator,mosquito_data_split, train_model_ml
import deepdish as dd
from joblib import dump, load
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, log_loss
from sklearn.model_selection import cross_val_score
seed = 42
np.random.seed(seed=seed)

In [ ]:
splitting = "random"
data_setting = "psdHQ"
model_setting = "knn"

assert splitting in ['random','randomcv','custom'], "Wrong splitting method given."
assert data_setting in ['raw','psd_dB','psdHQ'], "Wrong data settting given."
assert model_setting in ['knn','randomforest','xgboost']

data = Dataset('Wingbeats')
print(data.target_classes)

In [ ]:
print(f'SPLITTING DATA {splitting}')
X_train, X_val, X_test, y_train, y_val, y_test = mosquito_data_split(splitting=splitting, dataset=data)

In [ ]:
x_test = make_df_parallel(names=X_test, setting=data_setting).values

## if "RANDOM"

In [ ]:
results = {}
X_train.extend(X_val)
y_train.extend(y_val)
x_train = make_df_parallel(names=X_train, setting=data_setting).values
x_val = make_df_parallel(names=X_val, setting=data_setting).values

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
import multiprocessing
n_cpus = multiprocessing.cpu_count()

In [ ]:
estimator = KNeighborsClassifier(n_neighbors=11, weights='uniform',metric='manhattan', n_jobs=min(8,n_cpus))
parameters = {'n_neighbors':('7','9','11','13','15','17'), 'weights':('uniform','distance'), 'metric': ('manhattan','euclidean')}

In [ ]:
clf = GridSearchCV(estimator, parameters)

In [ ]:
cvfolds = 5
cv_results = cross_validate(estimator, x_train, y_train, cv=cvfolds, 
                            return_estimator=True, 
                            return_train_score=True, 
                            scoring=make_scorer(balanced_accuracy_score),
                            verbose=1, 
                            n_jobs=min(8, n_cpus)) 